In [4]:
# -*- coding: UTF-8 -*-

# 执行环境 Py3.
# 修改日期 2018/08/04
# 交叉验证lightGBM训练

import os, sys, codecs, gc
import pandas as pd
import numpy as np

from tqdm import tqdm, tqdm_notebook
from sklearn.externals.joblib import Parallel, delayed
from sklearn.metrics import f1_score, log_loss, classification_report
from sklearn.model_selection import StratifiedKFold

# 训练集的原始标签
train_label = pd.read_csv('../input/train_label.csv')

# API的名称
api = codecs.open('./api.csv', 'r').readlines()
api = [x[:-1] for x in api]

apidict2 = {}
for i, apicode in enumerate(api):
    apidict2[apicode] = 0

def fidfeat_single(id, path, part):
    if id % 100 == 0:
        print(id)
    
    df = pd.read_hdf(path + str(id) + '.hdf', part)
    
    apidict = apidict2.copy()
    apidict['file_id'] = id
     
    for rows in df['api'].value_counts().reset_index().iterrows():
        apidict[rows[1]['index']] = rows[1]['api']
       
    apidict['api_count'] = df['api'].nunique()
    apidict['tid_count'] = df['tid'].nunique()
    
    apidict['return_value_count'] = df['return_value'].nunique()
    apidict['return_value=0'] = sum(df['return_value'] == 0) / df.shape[0]
    apidict['return_value!=0'] = sum(df['return_value'] != 0) / df.shape[0]
    apidict['return_value==1'] = sum(df['return_value'] == 1) / df.shape[0]
    apidict['return_value=-1'] = sum(df['return_value'] == -1) / df.shape[0]
    apidict['return_value_max'] = df['return_value'].max()
    apidict['return_value_min'] = df['return_value'].min()
    apidict['return_value_std'] = df['return_value'].std()
    
    apidict['tid_first_value'] = df.groupby('tid').first()['return_value'].mean()
    apidict['tid_first_value!=0'] = sum(df.groupby('tid').first()['return_value'] != 0)
    apidict['tid_last_value'] = df.groupby('tid').last()['return_value'].mean()
    apidict['tid_last_value!=0'] = sum(df.groupby('tid').last()['return_value'] != 0)
    
    # Behaviour: File, Process, Memory, Register, Network, Service, Other.
    
    # 注册表信息，注册表修改信息
    reg_cols = ['RegOpenKeyExW', 'RegQueryValueExW', 'RegCloseKey', 'RegOpenKeyExA', 'RegQueryValueExA',
               'RegEnumKeyExW', 'RegQueryInfoKeyW', 'RegEnumValueW', 'RegEnumKeyW', 'RegCreateKeyExW',
               'RegSetValueExW', 'RegEnumValueA', 'RegDeleteValueW', 'RegCreateKeyExA', 'RegEnumKeyExA',
               'RegSetValueExA', 'RegDeleteValueA', 'RegDeleteKeyW', 'RegQueryInfoKeyA', 'RegDeleteKeyA']
    
    regalter_cols = ['RegCreateKeyExW', 'RegSetValueExW', 'RegDeleteValueW', 'RegCreateKeyExA',
               'RegSetValueExA', 'RegDeleteValueA', 'RegDeleteKeyW', 'RegDeleteKeyA']
    
    apidict['reg_info'] = int(df[df['api'].isin(reg_cols)].shape[0] > 1)
    apidict['reg_info_ratio'] = df[df['api'].isin(reg_cols)].shape[0] / (df.shape[0] + 1)
    
    apidict['reg_infoalter'] = int(df[df['api'].isin(regalter_cols)].shape[0] > 1)
    apidict['reg_infoalter_ratio'] = df[df['api'].isin(regalter_cols)].shape[0] / (df[df['api'].isin(reg_cols)].shape[0]+1)
    
    # 网络信息
    network_cols = ['InternetCrackUrlA', 'InternetSetOptionA', 'InternetGetConnectedState', 'InternetOpenW',
                   'InternetSetStatusCallback', 'InternetConnectW', 'InternetQueryOptionA', 'InternetCloseHandle',
                   'InternetOpenA', 'InternetConnectA', 'InternetOpenUrlA', 'InternetReadFile',
                   'InternetGetConnectedStateExW', 'InternetGetConnectedStateExA', 'InternetWriteFile']
    apidict['network_info'] = int(df[df['api'].isin(network_cols)].shape[0] > 1)
    
    # 内存信息
    memory_cols = ['NtAllocateVirtualMemory', 'NtFreeVirtualMemory', 'NtProtectVirtualMemory', 'WriteProcessMemory',
                  'ReadProcessMemory', 'NtReadVirtualMemory', 'CryptProtectMemory', 'CryptUnprotectMemory', 'NtWriteVirtualMemory']
    apidict['memory_info'] = int(df[df['api'].isin(memory_cols)].shape[0] > 1)
    
    # 文件信息
    file_cols = ['NtCreateFile', 'NtWriteFile', 'NtQueryAttributesFile', 'GetFileVersionInfoSizeW', 'GetFileVersionInfoW',
                'NtSetInformationFile', 'NtDeviceIoControlFile', 'NtOpenFile', 'FindFirstFileExW', 'GetFileAttributesW',
                'DeleteFileW', 'CopyFileA', 'SetFilePointer', 'NtReadFile', 'GetFileType', 'SetFileTime',
                'CopyFileW', 'MoveFileWithProgressW', 'CopyFileExW', 'NtDeleteFile']
    filealter_cols = ['NtCreateFile', 'NtWriteFile',
                'DeleteFileW', 'CopyFileA', 'SetFilePointer', 'SetFileTime',
                'CopyFileW', 'MoveFileWithProgressW', 'CopyFileExW', 'NtDeleteFile']
    
    apidict['file_info'] = int(df[df['api'].isin(file_cols)].shape[0] > 1)
    apidict['filealter_info'] = int(df[df['api'].isin(filealter_cols)].shape[0] > 1)
    apidict['filealter_ratio'] = apidict['filealter_info'] / (apidict['file_info'] + 1)
    
    # 进程信息
    thread_cols = ['CreateThread', 'Thread32First', 'Thread32Next', 'NtResumeThread', 'NtCreateThreadEx',
                   'NtOpenThread', 'NtTerminateThread', 'NtSuspendThread', 'NtGetContextThread'
                   'CreateRemoteThread', 'NtQueueApcThread', 'RtlCreateUserThread', 'NtSetContextThread',
                   'CreateRemoteThreadEx', 'NtCreateThread']
    apidict['thread_info'] = int(df[df['api'].isin(thread_cols)].shape[0] > 1)
    apidict['thread_ratio'] = df['api'].isin(thread_cols).iloc[-10:].sum() / 10
    
    # 服务信息
    service_cols = ['OpenServiceA', 'CreateServiceA', 'StartServiceA', 'CreateServiceW', 'StartServiceW',
                    'ControlService', 'DeleteService']
    apidict['service_info'] = int(df[df['api'].isin(reg_cols)].shape[0] > 1)
    
    # DLL信息
    dll_cols = ['LdrLoadDll', 'LdrUnloadDll', 'LdrGetDllHandle']
    apidict['dll_info'] = int(df[df['api'].isin(dll_cols)].shape[0] > 1)
    
    # 加密信息
    crypt_cols = ['CryptAcquireContextW', 'CryptProtectMemory', 'CryptUnprotectMemory', 'CryptHashData',
                 'CryptAcquireContextA', 'CryptEncrypt', 'CryptExportKey', 'CryptCreateHash', 'CryptDecodeObjectEx',
                 'CryptProtectData', 'CryptDecrypt', 'CryptUnprotectData']
    apidict['crypt_info'] = int(df[df['api'].isin(crypt_cols)].shape[0] > 1)
    
    # 证书信息
    cert_cols = ['CertCreateCertificateContext', 'CertOpenSystemStoreA', 'CertOpenSystemStoreW', 'CertOpenStore',
                'CertControlStore']
    apidict['cert_info'] = int(df[df['api'].isin(cert_cols)].shape[0] > 1)
    
    # COM信息
    com_cols = ['CoCreateInstance', 'CoCreateInstanceEx', 'CoGetClassObject', 'CoInitializeEx', 'CoInitializeSecurity',
               'CoUninitialize', 'ControlService']
    apidict['com_info'] = int(df[df['api'].isin(com_cols)].shape[0] > 1)
    
    # Find信息
    find_cols = ['FindResourceExW', 'FindResourceA', 'FindFirstFileExW', 'FindWindowA', 'FindResourceW', 
                'FindWindowW', 'FindResourceExA', 'FindWindowExW', 'FindFirstFileExA', 'FindWindowExA']
    apidict['find_info'] = int(df[df['api'].isin(find_cols)].shape[0] > 1)
    
    # Ldr 信息
    ldr_cols  = ['LdrLoadDll', 'LdrGetProcedureAddress', 'LdrUnloadDll', 'LdrGetDllHandle']
    apidict['ldr_info'] = int(df[df['api'].isin(ldr_cols)].shape[0] > 1)
    apidict['ldr_ratio'] = df['api'].isin(ldr_cols).iloc[-10:].sum() / 10
    
    # Nt 信息
    # TODO

    apidict['type_info1'] = apidict['network_info'] * apidict['service_info'] * apidict['reg_info']
    apidict['type_info2'] = apidict['ldr_info'] * apidict['file_info']
    apidict['type_info3'] = apidict['ldr_info'] * apidict['thread_info'] * apidict['memory_info']
    apidict['type_info4'] = apidict['crypt_info'] * apidict['find_info'] * apidict['network_info']
    
    return apidict

# df1 label-0
# df2 label-x
def fidfeat_augment(id, path, part):
    df1 = pd.read_hdf(path[0] + str(id[0]) + '.hdf', part[0])
    df2 = pd.read_hdf(path[1] + str(id[1]) + '.hdf', part[1])
    
    df1_tmp1 = df1.iloc[: int(df1.shape[0]*np.random.uniform(0, 0.2))]
    df1_tmp2 = df1.iloc[-int(df1.shape[0]*np.random.uniform(0, 0.2)):]
    df = pd.concat([df1_tmp1, df2, df1_tmp2])
    
    apidict = apidict2.copy()
    apidict['file_id'] = id[1]
     
    for rows in df['api'].value_counts().reset_index().iterrows():
        apidict[rows[1]['index']] = rows[1]['api']
       
    apidict['api_count'] = df['api'].nunique()
    apidict['tid_count'] = df['tid'].nunique()
    
    apidict['return_value_count'] = df['return_value'].nunique()
    apidict['return_value=0'] = sum(df['return_value'] == 0) / df.shape[0]
    apidict['return_value!=0'] = sum(df['return_value'] != 0) / df.shape[0]
    apidict['return_value==1'] = sum(df['return_value'] == 1) / df.shape[0]
    apidict['return_value=-1'] = sum(df['return_value'] == -1) / df.shape[0]
    apidict['return_value_max'] = df['return_value'].max()
    apidict['return_value_min'] = df['return_value'].min()
    apidict['return_value_std'] = df['return_value'].std()
    
    apidict['tid_first_value'] = df.groupby('tid').first()['return_value'].mean()
    apidict['tid_first_value!=0'] = sum(df.groupby('tid').first()['return_value'] != 0)
    apidict['tid_last_value'] = df.groupby('tid').last()['return_value'].mean()
    apidict['tid_last_value!=0'] = sum(df.groupby('tid').last()['return_value'] != 0)
    
    # Behaviour: File, Process, Memory, Register, Network, Service, Other.
    
    # 注册表信息，注册表修改信息
    reg_cols = ['RegOpenKeyExW', 'RegQueryValueExW', 'RegCloseKey', 'RegOpenKeyExA', 'RegQueryValueExA',
               'RegEnumKeyExW', 'RegQueryInfoKeyW', 'RegEnumValueW', 'RegEnumKeyW', 'RegCreateKeyExW',
               'RegSetValueExW', 'RegEnumValueA', 'RegDeleteValueW', 'RegCreateKeyExA', 'RegEnumKeyExA',
               'RegSetValueExA', 'RegDeleteValueA', 'RegDeleteKeyW', 'RegQueryInfoKeyA', 'RegDeleteKeyA']
    
    regalter_cols = ['RegCreateKeyExW', 'RegSetValueExW', 'RegDeleteValueW', 'RegCreateKeyExA',
               'RegSetValueExA', 'RegDeleteValueA', 'RegDeleteKeyW', 'RegDeleteKeyA']
    
    apidict['reg_info'] = int(df[df['api'].isin(reg_cols)].shape[0] > 1)
    apidict['reg_info_ratio'] = df[df['api'].isin(reg_cols)].shape[0] / (df.shape[0] + 1)
    
    apidict['reg_infoalter'] = int(df[df['api'].isin(regalter_cols)].shape[0] > 1)
    apidict['reg_infoalter_ratio'] = df[df['api'].isin(regalter_cols)].shape[0] / (df[df['api'].isin(reg_cols)].shape[0]+1)
    
    # 网络信息
    network_cols = ['InternetCrackUrlA', 'InternetSetOptionA', 'InternetGetConnectedState', 'InternetOpenW',
                   'InternetSetStatusCallback', 'InternetConnectW', 'InternetQueryOptionA', 'InternetCloseHandle',
                   'InternetOpenA', 'InternetConnectA', 'InternetOpenUrlA', 'InternetReadFile',
                   'InternetGetConnectedStateExW', 'InternetGetConnectedStateExA', 'InternetWriteFile']
    apidict['network_info'] = int(df[df['api'].isin(network_cols)].shape[0] > 1)
    
    # 内存信息
    memory_cols = ['NtAllocateVirtualMemory', 'NtFreeVirtualMemory', 'NtProtectVirtualMemory', 'WriteProcessMemory',
                  'ReadProcessMemory', 'NtReadVirtualMemory', 'CryptProtectMemory', 'CryptUnprotectMemory', 'NtWriteVirtualMemory']
    apidict['memory_info'] = int(df[df['api'].isin(memory_cols)].shape[0] > 1)
    
    # 文件信息
    file_cols = ['NtCreateFile', 'NtWriteFile', 'NtQueryAttributesFile', 'GetFileVersionInfoSizeW', 'GetFileVersionInfoW',
                'NtSetInformationFile', 'NtDeviceIoControlFile', 'NtOpenFile', 'FindFirstFileExW', 'GetFileAttributesW',
                'DeleteFileW', 'CopyFileA', 'SetFilePointer', 'NtReadFile', 'GetFileType', 'SetFileTime',
                'CopyFileW', 'MoveFileWithProgressW', 'CopyFileExW', 'NtDeleteFile']
    filealter_cols = ['NtCreateFile', 'NtWriteFile',
                'DeleteFileW', 'CopyFileA', 'SetFilePointer', 'SetFileTime',
                'CopyFileW', 'MoveFileWithProgressW', 'CopyFileExW', 'NtDeleteFile']
    
    apidict['file_info'] = int(df[df['api'].isin(file_cols)].shape[0] > 1)
    apidict['filealter_info'] = int(df[df['api'].isin(filealter_cols)].shape[0] > 1)
    apidict['filealter_ratio'] = apidict['filealter_info'] / (apidict['file_info'] + 1)
    
    # 进程信息
    thread_cols = ['CreateThread', 'Thread32First', 'Thread32Next', 'NtResumeThread', 'NtCreateThreadEx',
                   'NtOpenThread', 'NtTerminateThread', 'NtSuspendThread', 'NtGetContextThread'
                   'CreateRemoteThread', 'NtQueueApcThread', 'RtlCreateUserThread', 'NtSetContextThread',
                   'CreateRemoteThreadEx', 'NtCreateThread']
    apidict['thread_info'] = int(df[df['api'].isin(thread_cols)].shape[0] > 1)
    apidict['thread_ratio'] = df['api'].isin(thread_cols).iloc[-10:].sum() / 10
    
    # 服务信息
    service_cols = ['OpenServiceA', 'CreateServiceA', 'StartServiceA', 'CreateServiceW', 'StartServiceW',
                    'ControlService', 'DeleteService']
    apidict['service_info'] = int(df[df['api'].isin(reg_cols)].shape[0] > 1)
    
    # DLL信息
    dll_cols = ['LdrLoadDll', 'LdrUnloadDll', 'LdrGetDllHandle']
    apidict['dll_info'] = int(df[df['api'].isin(dll_cols)].shape[0] > 1)
    
    # 加密信息
    crypt_cols = ['CryptAcquireContextW', 'CryptProtectMemory', 'CryptUnprotectMemory', 'CryptHashData',
                 'CryptAcquireContextA', 'CryptEncrypt', 'CryptExportKey', 'CryptCreateHash', 'CryptDecodeObjectEx',
                 'CryptProtectData', 'CryptDecrypt', 'CryptUnprotectData']
    apidict['crypt_info'] = int(df[df['api'].isin(crypt_cols)].shape[0] > 1)
    
    # 证书信息
    cert_cols = ['CertCreateCertificateContext', 'CertOpenSystemStoreA', 'CertOpenSystemStoreW', 'CertOpenStore',
                'CertControlStore']
    apidict['cert_info'] = int(df[df['api'].isin(cert_cols)].shape[0] > 1)
    
    # COM信息
    com_cols = ['CoCreateInstance', 'CoCreateInstanceEx', 'CoGetClassObject', 'CoInitializeEx', 'CoInitializeSecurity',
               'CoUninitialize', 'ControlService']
    apidict['com_info'] = int(df[df['api'].isin(com_cols)].shape[0] > 1)
    
    # Find信息
    find_cols = ['FindResourceExW', 'FindResourceA', 'FindFirstFileExW', 'FindWindowA', 'FindResourceW', 
                'FindWindowW', 'FindResourceExA', 'FindWindowExW', 'FindFirstFileExA', 'FindWindowExA']
    apidict['find_info'] = int(df[df['api'].isin(find_cols)].shape[0] > 1)
    
    # Ldr 信息
    ldr_cols  = ['LdrLoadDll', 'LdrGetProcedureAddress', 'LdrUnloadDll', 'LdrGetDllHandle']
    apidict['ldr_info'] = int(df[df['api'].isin(ldr_cols)].shape[0] > 1)
    apidict['ldr_ratio'] = df['api'].isin(ldr_cols).iloc[-10:].sum() / 10
    
    # Nt 信息
    # TODO

    apidict['type_info1'] = apidict['network_info'] * apidict['service_info'] * apidict['reg_info']
    apidict['type_info2'] = apidict['ldr_info'] * apidict['file_info']
    apidict['type_info3'] = apidict['ldr_info'] * apidict['thread_info'] * apidict['memory_info']
    apidict['type_info4'] = apidict['crypt_info'] * apidict['find_info'] * apidict['network_info']
    
    return apidict
    
params = {
    'learning_rate': 0.05,
    'min_child_samples': 10,
    'max_depth': -1, 
    'lambda_l1': 6,
    'boosting': 'gbdt', 
    'objective': 'multiclass', 
    'metric': 'multi_logloss',
    'num_class': 6,
    'feature_fraction': .7,
    'bagging_fraction': .7,
    'seed': 99,
    'num_threads': 40,
    'verbose': 0
}

train_loss, val_loss = [], []
NFOLD = 6
skf = StratifiedKFold(n_splits = NFOLD, random_state = 1, shuffle = True)
train_pred = np.zeros((116624, NFOLD))
test_pred = np.zeros((53093, NFOLD))

# 单个fileid特征
# train_feat = Parallel(n_jobs=20)(delayed(fidfeat_single)(i, '../input/train/', 'train') for i in range(116624))
# test_feat = Parallel(n_jobs=20)(delayed(fidfeat_single)(i, '../input/test/', 'test') for i in range(53093))

for i, (tr_idx, val_idx) in enumerate(skf.split(train_label, train_label['label'])):
    print('Fold', i+1)

# train_pred /= NFOLD
# test_pred /= NFOLD


Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6


In [7]:
train_feat = Parallel(n_jobs=10)(delayed(fidfeat_single)(i, '../input/train/', 'train') for i in range(116624))

0


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [13]:
train_df = pd.DataFrame(train_feat)
test_df = pd.DataFrame(test_feat)

In [12]:
test_feat = Parallel(n_jobs=10)(delayed(fidfeat_single)(i, '../input/test/', 'test') for i in range(53093))

0


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [16]:
test_df['file_id'] == range(53093)

0        True
1        True
2        True
3        True
4        True
5        True
6        True
7        True
8        True
9        True
10       True
11       True
12       True
13       True
14       True
15       True
16       True
17       True
18       True
19       True
20       True
21       True
22       True
23       True
24       True
25       True
26       True
27       True
28       True
29       True
         ... 
53063    True
53064    True
53065    True
53066    True
53067    True
53068    True
53069    True
53070    True
53071    True
53072    True
53073    True
53074    True
53075    True
53076    True
53077    True
53078    True
53079    True
53080    True
53081    True
53082    True
53083    True
53084    True
53085    True
53086    True
53087    True
53088    True
53089    True
53090    True
53091    True
53092    True
Name: file_id, Length: 53093, dtype: bool

In [17]:
import lightgbm as lgb

In [175]:
    
params = {
    'learning_rate': 0.05,
    'min_child_samples': 10,
    'max_depth': 4, 
    'boosting': 'gbdt',
    'max_bin': 5,
    'objective': 'multiclass', 
    # 'metric': 'multi_logloss',
    'num_class': 6,
    'feature_fraction': .7,
    'bagging_fraction': .7,
    'seed': 99,
    'num_threads': 10,
    'verbose': 0
}

clf = lgb.train(params, train_set = lgb.Dataset(train_df.drop(['file_id'], axis=1), train_label['label']), 
                num_boost_round=150)

In [176]:
pred = clf.predict(test_df.drop('file_id', axis=1))

In [65]:
pred = pd.DataFrame(pred)

In [177]:
pd.DataFrame(np.argmax(pred, axis=1))[0].value_counts()

0    51527
5     1216
3      186
1      160
2        4
Name: 0, dtype: int64

In [46]:
unique, counts = np.unique(pred, return_counts=True)

In [48]:
unique

array([9.12056634e-05, 9.12120163e-05, 9.12132794e-05, ...,
       9.96215299e-01, 9.96273720e-01, 9.96849495e-01])

In [150]:
params = {
    'learning_rate': 0.1,
    'min_child_samples': 2,
    'max_depth': 6, 
    'boosting': 'gbdt',
    # 'max_bin': 5,
    'objective': 'multiclass', 
    'metric': 'multi_logloss',
    'num_class': 6,
    # 'feature_fraction': .7,
#     'bagging_fraction': .7,
    'seed': 99,
    'num_threads': 10,
    'verbose': 0,
    'is_training_metric': 'True'
}


NFOLD = 6
skf = StratifiedKFold(n_splits = NFOLD, random_state = 1, shuffle = True)
test_pred = np.zeros((53093, 6))

for i, (tr_idx, val_idx) in enumerate(skf.split(train_df, train_label['label'])):
    tr_x, val_x = train_df.drop(['file_id'], axis=1).iloc[tr_idx], train_df.drop(['file_id'], axis=1).iloc[val_idx]
    tr_y, val_y = train_label['label'].iloc[tr_idx].values, train_label['label'].iloc[val_idx].values
    
    print(tr_x.shape, val_x.shape)
    
    tr_dataset = lgb.Dataset(tr_x, tr_y)
    val_dataset = lgb.Dataset(val_x, val_y)
    
    clf = lgb.train(params, train_set = tr_dataset, valid_sets= [tr_dataset, val_dataset],
                    valid_names = ['train', 'valid'], 
                verbose_eval=200, early_stopping_rounds=50, num_boost_round=600)
    # break
    print(log_loss(tr_y, clf.predict(tr_x)))
    print(log_loss(val_y, clf.predict(val_x)))
    
    test_pred += clf.predict(test_df.drop(['file_id'], axis=1))
#     gbm.predict(X_test, num_iteration=gbm.best_iteration)

(97185, 345) (19439, 345)
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[142]	train's multi_logloss: 0.00589455	valid's multi_logloss: 0.027768
0.005894550903284722
0.027767990663223083
(97186, 345) (19438, 345)
Training until validation scores don't improve for 50 rounds.
[200]	train's multi_logloss: 0.00387263	valid's multi_logloss: 0.0271939
Early stopping, best iteration is:
[158]	train's multi_logloss: 0.00529796	valid's multi_logloss: 0.0268563
0.005297955397094447
0.02685630853250065
(97186, 345) (19438, 345)
Training until validation scores don't improve for 50 rounds.
[200]	train's multi_logloss: 0.00350763	valid's multi_logloss: 0.0272207
Early stopping, best iteration is:
[162]	train's multi_logloss: 0.00472653	valid's multi_logloss: 0.0271004
0.004726529407483025
0.027100411419432743
(97186, 345) (19438, 345)
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[141]	train's multi

In [28]:
# 训练集的原始标签
train_label = pd.read_csv('../input/train_label.csv')

In [29]:
train_label2 = pd.concat([train_label[train_label['label'] == 0].sample(frac=0.3), 
           train_label[train_label['label'] != 0]], axis=0)

train_label2.sort_values(by='file_id', inplace=True)

In [33]:
train_label2['file_id'].values

array([     1,      2,      3, ..., 116617, 116621, 116622])

In [36]:
df1 = pd.read_hdf('../input/train/1000.hdf', 'train')

In [37]:
df1

,file_id,label,api,tid,return_value,index
3486072,1000,0,GetSystemTimeAsFileTime,2620,0,0
3486073,1000,0,SetUnhandledExceptionFilter,2620,0,1
3486074,1000,0,CoInitializeEx,2620,0,2
3486075,1000,0,LdrLoadDll,2620,-1073741515,3
3486076,1000,0,LdrLoadDll,2620,0,4
3486077,1000,0,NtQuerySystemInformation,2620,0,5
3486078,1000,0,NtQueryKey,2620,0,6
3486079,1000,0,NtOpenKeyEx,2620,0,7
3486080,1000,0,NtQueryValueKey,2620,-1073741772,8
3486081,1000,0,NtClose,2620,0,9
